In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# load data
dataframe = pd.read_csv('data.csv')
# remove the features we don't care about
dataframe = dataframe.drop(['index','price','sq_price'], axis=1) 
dataframe = dataframe[0:10]
dataframe

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


In [3]:
# add labels
dataframe.loc[:,('y1')] = [1,1,1,0,0,1,0,1,1,1] # 1 is good, 0 is bad 
dataframe.loc[:,('y2')] = dataframe['y1'] == 0 # y2 is opposite of y1
# transfer from 'ture/false' to '0/1'
dataframe.loc[:,('y2')] = dataframe['y2'].astype(int) 
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [4]:
# prepare train data
inputX = dataframe.loc[:, ['area', 'bathrooms']].as_matrix()
inputY = dataframe.loc[:, ['y1', 'y2']].as_matrix()
inputX

array([[  2.10400000e+03,   3.00000000e+00],
       [  1.60000000e+03,   3.00000000e+00],
       [  2.40000000e+03,   3.00000000e+00],
       [  1.41600000e+03,   2.00000000e+00],
       [  3.00000000e+03,   4.00000000e+00],
       [  1.98500000e+03,   4.00000000e+00],
       [  1.53400000e+03,   3.00000000e+00],
       [  1.42700000e+03,   3.00000000e+00],
       [  1.38000000e+03,   3.00000000e+00],
       [  1.49400000e+03,   3.00000000e+00]])

In [19]:
# set hyperparameters
learning_rate = 0.00001
training_epochs = 2000
display_step = 100
n_samples = inputY.size

In [49]:
# computation graph
x = tf.placeholder(tf.float32, (None,2) ) # for input, None means any numbers of examples, 2 means tow features

# create weights
W = tf.Variable(tf.zeros([2,2])) # Variables for parameter needing update

# add biases
b = tf.Variable(tf.zeros([2]))

# compute X * W + b
y_values = tf.add(tf.matmul(x,W), b)

# apply softmax(activation function)
y = tf.nn.softmax(y_values)

# feed in labels
y_ = tf.placeholder(tf.float32, (None,2) )



In [50]:
# perform training
cost = tf.reduce_sum(tf.pow(y_ - y, 2)) / (2 * n_samples)  # use cost function MSE
# gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [51]:
# initialize variables and tensorflow session
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [52]:
# training loop
for i in range(training_epochs):
    sess.run(optimizer, feed_dict = {x:inputX, y_: inputY})
    # write out logs of training
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict = {x:inputX, y_:inputY})
        print ("Training step:" , '%04d' % (i), "cost=", "{:.9f}".format(cc) )
        

print ("Optimization Finished!")
training_cost = sess.run(cost, feed_dict={x: inputX, y_: inputY})
print ("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')
      

Training step: 0000 cost= 0.134503588
Training step: 0100 cost= 0.112963334
Training step: 0200 cost= 0.112963378
Training step: 0300 cost= 0.112963438
Training step: 0400 cost= 0.112963483
Training step: 0500 cost= 0.112963557
Training step: 0600 cost= 0.112963602
Training step: 0700 cost= 0.112963654
Training step: 0800 cost= 0.112963714
Training step: 0900 cost= 0.112963758
Training step: 1000 cost= 0.112963818
Training step: 1100 cost= 0.112963870
Training step: 1200 cost= 0.112963915
Training step: 1300 cost= 0.112963974
Training step: 1400 cost= 0.112964012
Training step: 1500 cost= 0.112964079
Training step: 1600 cost= 0.112964131
Training step: 1700 cost= 0.112964176
Training step: 1800 cost= 0.112964250
Training step: 1900 cost= 0.112964295
Optimization Finished!
Training cost= 0.110957 W= [[ 0.00014077 -0.00014077]
 [ 0.00045578 -0.00045578]] b= [  9.51521433e-05  -9.51521361e-05] 



In [42]:
sess.run(y, feed_dict= {x: inputX})

array([[ 0.64457554,  0.3554244 ],
       [ 0.61144364,  0.38855633],
       [ 0.66342938,  0.33657062],
       [ 0.59884864,  0.40115133],
       [ 0.70024157,  0.29975849],
       [ 0.63707429,  0.36292577],
       [ 0.60702002,  0.39298001],
       [ 0.59981126,  0.40018877],
       [ 0.59663081,  0.40336913],
       [ 0.60433042,  0.39566955]], dtype=float32)

In [ ]:
# we can see above data meaning all good buy
# but we only have 7/10 good by in inputX
# how to improve? add a hidden layer
